In [2]:
from transformers import pipeline
import warnings

warnings.filterwarnings("ignore")

print("Initializing the Smart Resume Parser...")

# We use a specific BERT model fine-tuned for NER (Named Entity Recognition)
# aggregation_strategy="simple" is crucial: it combines sub-tokens (like "Stan" + "##ford") into whole words.
ner_pipeline = pipeline("ner",
                        model="dbmdz/bert-large-cased-finetuned-conll03-english",
                        aggregation_strategy="simple")

print("✅ Model Loaded Successfully!")

Initializing the Smart Resume Parser...


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


✅ Model Loaded Successfully!


In [3]:
def parse_resume(text):
    """
    Analyzes a resume text and extracts structured entities.
    Returns a dictionary of found entities.
    """
    print("🔍 Scanning resume text...")
    results = ner_pipeline(text)

    # Use sets to avoid duplicates (e.g. if 'Google' appears 5 times)
    profile = {
        "Candidate Name": set(),
        "Companies": set(),
        "Universities/Locations": set()
    }

    for entity in results:
        # Filter for high confidence to avoid errors
        if entity['score'] < 0.85:
            continue

        entity_type = entity['entity_group']
        word = entity['word'].strip()

        # Categorize based on BERT's entity groups (PER, ORG, LOC)
        if entity_type == "PER":
            profile["Candidate Name"].add(word)
        elif entity_type == "ORG":
            profile["Companies"].add(word)
        elif entity_type == "LOC":
            profile["Universities/Locations"].add(word)

    return profile

print("✅ Function 'parse_resume' is ready.")

✅ Function 'parse_resume' is ready.


In [4]:
resume_text = """
Sophia R. Turner is a Lead Data Scientist with 5 years of experience.
She previously worked at Microsoft and Tesla on autonomous vehicle systems.
Sophia obtained her Master's degree in Artificial Intelligence from Stanford University
and her Bachelor's from MIT (Massachusetts Institute of Technology).
She is currently based in San Francisco but is open to remote roles in New York.
"""

print(f"📄 Input Resume:\n{'-'*20}\n{resume_text.strip()}\n{'-'*20}")

📄 Input Resume:
--------------------
Sophia R. Turner is a Lead Data Scientist with 5 years of experience.
She previously worked at Microsoft and Tesla on autonomous vehicle systems.
Sophia obtained her Master's degree in Artificial Intelligence from Stanford University 
and her Bachelor's from MIT (Massachusetts Institute of Technology).
She is currently based in San Francisco but is open to remote roles in New York.
--------------------


In [5]:
extracted_data = parse_resume(resume_text)

print("\n" + "="*40)
print("🤖 SMART RESUME PARSER REPORT")
print("="*40)

# Helper to print nicely
def print_field(label, items):
    if not items:
        content = "None found"
    else:
        content = ", ".join(sorted(items))
    print(f"{label:<25} : {content}")

print_field("👤 Candidate Name", extracted_data["Candidate Name"])
print_field("🏢 Experience (Orgs)", extracted_data["Companies"])
print_field("📍 Education/Locations", extracted_data["Universities/Locations"])
print("="*40)

🔍 Scanning resume text...

🤖 SMART RESUME PARSER REPORT
👤 Candidate Name          : Sophia, Sophia R. Turner
🏢 Experience (Orgs)       : MIT, Massachusetts Institute of Technology, Microsoft, Stanford University, Tesla
📍 Education/Locations     : New York, San Francisco
